In [7]:
import pandas as pd
import numpy as np
import ensemble_trading_strategy as ets
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
df = pd.read_csv('../data/S & P 50/GOOGL.csv')
df.head()

,date,open,high,low,close,volume
0,2018-01-02 00:00:00.000000,52.651001,53.799000,52.651001,53.660500,31766000.0
1,2018-01-03 00:00:00.000000,53.696499,54.805000,53.671501,54.576000,31318000.0
2,2018-01-04 00:00:00.000000,54.854500,55.203999,54.713001,54.787998,26052000.0
3,2018-01-05 00:00:00.000000,55.172501,55.679001,55.090000,55.514500,30250000.0
4,2018-01-08 00:00:00.000000,55.549999,55.958000,55.500000,55.710499,24644000.0


In [26]:
X = ets.add_features(df)
X.head(10)

,open,high,low,close,volume,sma_16,ema_16,rsi_16,cci_16,atr,STOCHk_14_3_3,STOCHd_14_3_3,STOCHRSIk_16_14_3_3,STOCHRSId_16_14_3_3,wpr
33,54.638000,55.814499,54.500000,55.179501,33754000.0,54.921625,54.626231,53.146765,6.816099,1.318708,51.500782,44.859306,35.043782,30.688683,-46.970144
34,55.455002,56.810001,55.375500,55.687500,41486000.0,54.694344,54.751086,55.097203,41.309148,1.338196,56.286414,51.062018,39.636307,35.290979,-38.851927
35,55.958500,56.272999,55.257500,55.494999,28010000.0,54.483500,54.838606,54.185457,43.934166,1.318027,66.125307,57.970834,43.114389,39.264826,-40.719362
36,55.932999,56.450001,55.422001,56.404499,25292000.0,54.314344,55.022828,57.712081,75.168091,1.299900,78.860764,67.090828,48.095241,43.615312,-31.168301
37,56.592999,57.209999,56.466999,57.185001,30912000.0,54.195938,55.277202,60.495643,128.689272,1.269000,91.646824,78.877632,65.858891,52.356174,-0.339643
38,57.185001,57.212502,55.839500,55.875500,42964000.0,54.190656,55.347590,54.120276,96.535698,1.275500,91.891522,87.466370,76.576946,63.510359,-18.159609
39,56.099998,56.382500,55.150002,55.195999,49292000.0,54.320438,55.329755,51.137419,55.405069,1.272813,84.703977,89.414108,81.092659,74.509499,-27.388816
40,55.477001,55.563499,53.364498,53.570499,55338000.0,54.279750,55.122784,44.832352,-9.315547,1.330699,68.328225,81.641242,62.462029,73.377211,-49.466901
41,52.898998,54.344501,52.505501,54.207001,50162000.0,54.369531,55.015045,47.534244,-32.985470,1.362468,54.093180,69.041794,54.816010,66.123566,-40.821726
42,53.906502,55.058998,53.613499,54.737999,28648000.0,54.641562,54.982451,49.725310,-6.778412,1.367658,46.273560,56.231655,47.525860,54.934633,-33.609529


In [27]:
y = ets.get_target_signal_next_price(X)
y.tail()

1253    1
1254   -1
1255   -1
1256    1
1257    0
Name: signals, dtype: int64

In [28]:
X.columns

Index(['open', 'high', 'low', 'close', 'volume', 'sma_16', 'ema_16', 'rsi_16',
       'cci_16', 'atr', 'STOCHk_14_3_3', 'STOCHd_14_3_3',
       'STOCHRSIk_16_14_3_3', 'STOCHRSId_16_14_3_3', 'wpr'],
      dtype='object')

In [29]:
y.value_counts()

signals
 0    726
 1    262
-1    237
Name: count, dtype: int64

In [30]:
print(X.shape, y.shape)

(1225, 15) (1225,)


In [31]:
X_scaled = ets.data_preprocess(X)
X_scaled[:3]

array([[-1.03506216, -1.01845604, -1.01988969, -1.0190734 , -0.08368291,
        -1.02054956, -1.03175025, -0.01184051, -0.12261917, -0.81497293,
        -0.19297861, -0.44747906, -0.56780558, -0.73536419, -0.1374901 ],
       [-1.0085258 , -0.98642529, -0.991118  , -1.00255172,  0.40702112,
        -1.02794233, -1.02768231,  0.1745906 ,  0.19947059, -0.79545634,
        -0.02169734, -0.21734298, -0.42449871, -0.58537061,  0.13058442],
       [-0.99217209, -1.00370359, -0.99499586, -1.00881243, -0.4482204 ,
        -1.03480044, -1.02483084,  0.08744205,  0.22398253, -0.81565547,
         0.33044391,  0.03899135, -0.31596735, -0.4558588 ,  0.06891918]])

In [20]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib


In [32]:
# splitting the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, random_state=0)

# train with RFC model
rf_model = RandomForestClassifier(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

# making predictions with test data
predictions = rf_model.predict(X_test)

# saving the model
# joblib.dump(rf_model, 'random_forrest_classifier_on_return_signal.pkl')

# accuracy calculation
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy of RFC model: {accuracy}')

Accuracy of RFC model: 0.5895765472312704


In [33]:
from sklearn.tree import DecisionTreeClassifier
# train with Bagging Classifier
bag_model = BaggingClassifier(base_estimator=DecisionTreeClassifier()
                         ,n_estimators=100
                         ,max_samples=0.8
                         ,oob_score=True
                         ,random_state=10)
bag_model.fit(X_train, y_train)

# making predictions with test data
predictions = bag_model.predict(X_test)

# saving the model
# joblib.dump(bag_model, 'bagging_logistic_regression_next_price.pkl')

# accuracy calculation
accuracy = accuracy_score(y_test, predictions)
print(f'OOB Score {bag_model.oob_score_}')
print(f'Accuracy of Bagging with LR: {accuracy}')

/home/sohan/anaconda3/envs/env31/lib/python3.10/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


OOB Score 0.5686274509803921
Accuracy of Bagging with LR: 0.6026058631921825


In [34]:
ets.train_bag_model_with_lr(X_scaled, y)

/home/sohan/anaconda3/envs/env31/lib/python3.10/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/home/sohan/anaconda3/envs/env31/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/sohan/anaconda3/envs/env31/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or sca

Accuracy of Bagging with LR: 0.6058631921824105
